In [2]:
from faker import Faker
import random
import sqlite3
import pandas as pd

fake = Faker()

# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''CREATE TABLE Customers (
                    customer_id INTEGER PRIMARY KEY,
                    name TEXT,
                    shipping_addr TEXT,
                    contact_number TEXT,
                    current_flag INTEGER DEFAULT 1
                )''')

# Create Orders table
cursor.execute('''CREATE TABLE Orders (
                    order_id INTEGER PRIMARY KEY,
                    customer_id INTEGER,
                    order_date TEXT,
                    total_amount REAL NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
                )''')

# Create Order Items table
cursor.execute('''CREATE TABLE OrderItems (
                    item_id INTEGER PRIMARY KEY,
                    order_id INTEGER,
                    product_id INTEGER,
                    quantity INTEGER,
                    price REAL,
                    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
                    FOREIGN KEY (product_id) REFERENCES Products(product_id)
                )''')

# Create Products table
cursor.execute('''CREATE TABLE Products (
                    product_id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT
                )''')

# Generate customers
for _ in range(10):
    name = fake.name()
    shipping_addr = fake.address().replace("\n", ", ")
    contact_number = fake.phone_number()
    cursor.execute('''INSERT INTO Customers (name, shipping_addr, contact_number)
                      VALUES (?, ?, ?)''', (name, shipping_addr, contact_number))
    conn.commit()

# Generate products
for _ in range(10):
    name = fake.word()
    description = fake.sentence(nb_words=6)
    cursor.execute('''INSERT INTO Products (name, description)
                      VALUES (?, ?)''', (name, description))
    conn.commit()

# Generate orders and order items
for _ in range(100):
    customer_id = random.randint(1, 10)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    total_amount = round(random.uniform(10, 1000), 2)
    cursor.execute('''INSERT INTO Orders (customer_id, order_date, total_amount)
                      VALUES (?, ?, ?)''', (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid

    # Generate order items
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.randint(1, 10)
        quantity = random.randint(1, 10)
        price = round(random.uniform(10, 100), 2)
        cursor.execute('''INSERT INTO OrderItems (order_id, product_id, quantity, price)
                          VALUES (?, ?, ?, ?)''', (order_id, product_id, quantity, price))

    conn.commit()

# Close the database connection
conn.close()

In [8]:
# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Get orders in dataframe
cursor.execute("SELECT * FROM Orders")
orders = cursor.fetchall()
orders_df = pd.DataFrame(orders, columns=[column[0] for column in cursor.description])

# Get Customers in dataframe
cursor.execute("SELECT * FROM Customers")
customers = cursor.fetchall()
customers_df = pd.DataFrame(customers, columns=[column[0] for column in cursor.description])

# Get Products in dataframe
cursor.execute("SELECT * FROM Products")
products = cursor.fetchall()
products_df = pd.DataFrame(products, columns=[column[0] for column in cursor.description])

# Get OrderItems in dataframe
cursor.execute("SELECT * FROM OrderItems")
order_items = cursor.fetchall()
order_items_df = pd.DataFrame(order_items, columns=[column[0] for column in cursor.description])

# Close the database connection
conn.close()

In [9]:
orders_df.head()

,order_id,customer_id,order_date,total_amount
0,1,4,2022-06-22,412.66
1,2,6,2022-12-14,485.74
2,3,1,2023-01-25,268.05
3,4,4,2022-08-16,492.08
4,5,7,2021-09-18,416.61


In [10]:
customers_df.head()

,customer_id,name,shipping_addr,contact_number,current_flag
0,1,Richard Hoffman,"532 Aaron Passage Apt. 040, Thomashaven, SD 30965",277.566.6576,1
1,2,Jorge Burns,"5334 Joseph Plain Suite 479, North Brittanyber...",607-519-1516,1
2,3,Stephen Clark,"2236 Bolton Highway, East Shannonhaven, MI 43106",001-926-956-6630x535,1
3,4,Kim Mosley,"947 Johnson Isle Suite 344, West Kennethside, ...",292-853-4387,1
4,5,Cathy Hodge PhD,"6074 Woods Lake, Port Jessica, CO 05966",+1-928-623-0339,1


In [14]:
products_df.head()

,product_id,name,description
0,1,firm,Garden message else piece relationship.
1,2,out,Health marriage thank eight brother.
2,3,help,Reality discussion room against future.
3,4,both,Every audience within specific forget.
4,5,environment,Production crime send Congress can entire.


In [12]:
order_items_df.head()

,item_id,order_id,product_id,quantity,price
0,1,1,10,3,94.07
1,2,1,3,4,20.49
2,3,1,6,1,18.36
3,4,1,5,1,58.26
4,5,1,7,10,72.84
